## NLP PROJECT: MACHINE TRANSLATION 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10627536362974003862
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5113585898624615581
physical_device_desc: "device: XLA_CPU device"
]


**LOADING OF DATASET AND UNDERSTANDING OF DATA**

In [ ]:
def read_data(filename):
            file = open(filename, mode='rt', encoding='utf-8')
            text = file.read()
            file.close()
            return text

In [ ]:
def lines(text):
      sents = text.strip().split('\n')
      return sents

In [ ]:
data= read_data("/content/drive/My Drive/NMT/small_vocab_en")
english_sentences = lines(data)
data1= read_data("/content/drive/My Drive/NMT/small_vocab_fr")
french_sentences = lines(data1)


In [ ]:
for sample_i in range(5):
    print('English sample {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('French sample {}:  {}\n'.format(sample_i + 1, french_sentences[sample_i]))

English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .
French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

English sample 3:  california is usually quiet during march , and it is usually hot in june .
French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .

English sample 4:  the united states is sometimes mild during june , and it is cold in september .
French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .

English sample 5:  your least liked fruit is the grape , but my least liked is the apple .
French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .



In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


**TOKENIZATION**

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)b
    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
text_sentences = [
    "dogs are man's best friend.",
    'He decided to skiing on a frozen lake .',
    'Toddlers feeding raccoons.']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'dogs': 1, 'are': 2, "man's": 3, 'best': 4, 'friend': 5, 'he': 6, 'decided': 7, 'to': 8, 'skiing': 9, 'on': 10, 'a': 11, 'frozen': 12, 'lake': 13, 'toddlers': 14, 'feeding': 15, 'raccoons': 16}

Sequence 1 in x
  Input:  dogs are man's best friend.
  Output: [1, 2, 3, 4, 5]
Sequence 2 in x
  Input:  He decided to skiing on a frozen lake .
  Output: [6, 7, 8, 9, 10, 11, 12, 13]
Sequence 3 in x
  Input:  Toddlers feeding raccoons.
  Output: [14, 15, 16]


**PADDING**

In [ ]:
def pad(x, length=None):

    return pad_sequences(x, maxlen=length, padding='post')

In [ ]:
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 3 4 5]
  Output: [1 2 3 4 5 0 0 0]
Sequence 2 in x
  Input:  [ 6  7  8  9 10 11 12 13]
  Output: [ 6  7  8  9 10 11 12 13]
Sequence 3 in x
  Input:  [14 15 16]
  Output: [14 15 16  0  0  0  0  0]


In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [ ]:
def logits_to_text(logits, tokenizer):
    
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


**SIMPLE RNN MODEL**

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
       
    model = Sequential([LSTM(100,input_shape=(21,1),return_sequences=True),TimeDistributed(Dense(345,activation='softmax'))])
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.001),metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.summary()
simple_rnn_model.fit(tmp_x, preproc_french_sentences, epochs=10,batch_size=1024,validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 21, 100)           40800     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 345)           34845     
Total params: 75,645
Trainable params: 75,645
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 57s 524ms/step - loss: 3.1345 - accuracy: 0.4261 - val_loss: 2.4353 - val_accuracy: 0.4751
Epoch 2/10
108/108 [==============================] - 56s 522ms/step - loss: 2.1993 - accuracy: 0.5016 - val_loss: 1.9816 - val_accuracy: 0.5407
Epoch 3/10
108/108 [==============================] - 57s 529ms/step - loss: 1.8470 - accuracy: 0.5643 - val_loss: 1.7387 - val_accuracy: 0.5808
Epoch 4/10
108/108 [==============================] - 62s 576ms/step - loss: 

**RNN WITH EMBEDDING**

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 0.005
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(LSTM(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 256)           51200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 21, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout (Dropout)            (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           353625    
Total params: 1,193,305
Trainable params: 1,193,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 375s 3s/step - loss: 1.6237 - accuracy: 0.6345 - val_los

In [ ]:
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


BIDIRECTIONAL RNN

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
       learning_rate = 0.003
    
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 256)           51200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 21, 256)           525312    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           353625    
Total params: 1,193,305
Trainable params: 1,193,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 372s 3s/step - loss: 1.5595 - accuracy: 0.6441 - val_los

ENCODER AND DECODER MODEL

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 0.001
        
    model = Sequential()
    # Encoder
    model.add(LSTM(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               264192    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 21, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 256)           525312    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 1024)          263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 345)           353625    
Total params: 1,406,297
Trainable params: 1,406,297
Non-trainable params: 0
____________________________________________

EMBEDDING + BIDIRECTIONAL ENCODER model


In [ ]:
   def model_embie(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate = 0.003
    
    # Build the layers    
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(LSTM(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [ ]:
def embie_predictions(x, y, x_tk, y_tk):
    # TODO: Train neural network using model_final
    model = model_embie(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=25, validation_split=0.2)

    
embie_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 128)           25600     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 21, 256)           394240    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 512)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 512)           0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 345)          